In [1]:
# notebooks/map_system.ipynb

# RPGシミュレーター - マップシステム
# 
# このノートブックでは、RPGシミュレーターのマップシステムを実装します。

import json
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional, Any
from IPython.display import clear_output

# キャラクター定義を読み込む
%run notebooks/characters.ipynb

# マップタイルの定義
TILE_EMPTY = 0     # 何もない（通行可能）
TILE_WALL = 1      # 壁（通行不可）
TILE_GRASS = 2     # 草原（通行可能、エンカウント率高い）
TILE_FOREST = 3    # 森（通行可能、移動コスト高い）
TILE_WATER = 4     # 水（通行不可）
TILE_MOUNTAIN = 5  # 山（通行可能、移動コスト非常に高い）
TILE_ROAD = 6      # 道（通行可能、移動コスト低い）
TILE_TOWN = 7      # 町（通行可能、イベント発生）
TILE_DUNGEON = 8   # ダンジョン入口（通行可能、イベント発生）
TILE_CASTLE = 9    # 城（通行可能、イベント発生）

# タイルの表示色
TILE_COLORS = {
    TILE_EMPTY: '#f5f5dc',    # ベージュ
    TILE_WALL: '#808080',      # グレー
    TILE_GRASS: '#90ee90',     # ライトグリーン
    TILE_FOREST: '#228b22',    # フォレストグリーン
    TILE_WATER: '#4169e1',     # ロイヤルブルー
    TILE_MOUNTAIN: '#a0522d',  # ブラウン
    TILE_ROAD: '#f4a460',      # サンディブラウン
    TILE_TOWN: '#ff6347',      # トマト
    TILE_DUNGEON: '#800080',   # パープル
    TILE_CASTLE: '#ffd700'     # ゴールド
}

# タイルの日本語名
TILE_NAMES = {
    TILE_EMPTY: '平地',
    TILE_WALL: '壁',
    TILE_GRASS: '草原',
    TILE_FOREST: '森',
    TILE_WATER: '水',
    TILE_MOUNTAIN: '山',
    TILE_ROAD: '道',
    TILE_TOWN: '町',
    TILE_DUNGEON: 'ダンジョン',
    TILE_CASTLE: '城'
}

# 移動コスト（値が高いほど移動に時間/労力がかかる）
TILE_MOVEMENT_COST = {
    TILE_EMPTY: 1.0,
    TILE_WALL: float('inf'),  # 通行不可
    TILE_GRASS: 1.2,
    TILE_FOREST: 1.8,
    TILE_WATER: float('inf'),  # 通行不可
    TILE_MOUNTAIN: 3.0,
    TILE_ROAD: 0.8,
    TILE_TOWN: 1.0,
    TILE_DUNGEON: 1.0,
    TILE_CASTLE: 1.0
}

# エンカウント率（値が高いほど敵と遭遇しやすい）
TILE_ENCOUNTER_RATE = {
    TILE_EMPTY: 0.05,
    TILE_WALL: 0.0,
    TILE_GRASS: 0.15,
    TILE_FOREST: 0.2,
    TILE_WATER: 0.0,
    TILE_MOUNTAIN: 0.1,
    TILE_ROAD: 0.05,
    TILE_TOWN: 0.0,
    TILE_DUNGEON: 0.3,
    TILE_CASTLE: 0.0
}

# 移動方向の定義
DIRECTION_UP = 0
DIRECTION_RIGHT = 1
DIRECTION_DOWN = 2
DIRECTION_LEFT = 3

# 方向と座標変化の対応
DIRECTION_CHANGES = {
    DIRECTION_UP: (-1, 0),
    DIRECTION_RIGHT: (0, 1),
    DIRECTION_DOWN: (1, 0),
    DIRECTION_LEFT: (0, -1)
}

# マップ上の場所（ロケーション）クラス
class Location:
    """マップ上の特定の場所を表すクラス"""
    
    def __init__(self, name: str, description: str, loc_type: str, 
                 row: int, col: int, events: List[Dict] = None):
        """ロケーションの初期化
        
        Args:
            name: 場所の名前
            description: 場所の説明
            loc_type: 場所のタイプ（'town', 'dungeon', 'castle', etc）
            row: マップ上の行位置
            col: マップ上の列位置
            events: この場所で発生するイベントのリスト
        """
        self.name = name
        self.description = description
        self.loc_type = loc_type
        self.row = row
        self.col = col
        self.events = events or []
        self.visited = False
    
    def visit(self) -> List[Dict]:
        """この場所を訪れた時の処理
        
        Returns:
            発生するイベントのリスト
        """
        self.visited = True
        return self.events
    
    def to_dict(self) -> Dict:
        """ロケーション情報を辞書形式で取得
        
        Returns:
            ロケーション情報の辞書
        """
        return {
            'name': self.name,
            'description': self.description,
            'type': self.loc_type,
            'position': (self.row, self.col),
            'visited': self.visited
        }

# マップクラス
class GameMap:
    """RPGのマップを表すクラス"""
    
    def __init__(self, name: str, width: int, height: int):
        """マップの初期化
        
        Args:
            name: マップ名
            width: マップの幅
            height: マップの高さ
        """
        self.name = name
        self.width = width
        self.height = height
        self.tiles = np.zeros((height, width), dtype=int)
        self.locations = {}  # 座標をキー、Locationオブジェクトを値
        
    def set_tile(self, row: int, col: int, tile_type: int) -> bool:
        """特定の位置のタイルを設定
        
        Args:
            row: 行位置
            col: 列位置
            tile_type: タイルタイプ
            
        Returns:
            設定に成功したかどうか
        """
        if 0 <= row < self.height and 0 <= col < self.width:
            self.tiles[row, col] = tile_type
            return True
        return False
    
    def get_tile(self, row: int, col: int) -> int:
        """特定の位置のタイルを取得
        
        Args:
            row: 行位置
            col: 列位置
            
        Returns:
            タイルタイプ（範囲外の場合はTILE_WALL）
        """
        if 0 <= row < self.height and 0 <= col < self.width:
            return self.tiles[row, col]
        return TILE_WALL  # マップ外は壁として扱う
    
    def is_walkable(self, row: int, col: int) -> bool:
        """特定の位置が通行可能かどうか
        
        Args:
            row: 行位置
            col: 列位置
            
        Returns:
            通行可能かどうか
        """
        tile = self.get_tile(row, col)
        return TILE_MOVEMENT_COST[tile] < float('inf')
    
    def add_location(self, location: Location) -> bool:
        """マップにロケーションを追加
        
        Args:
            location: 追加するロケーション
            
        Returns:
            追加に成功したかどうか
        """
        key = (location.row, location.col)
        if 0 <= location.row < self.height and 0 <= location.col < self.width:
            self.locations[key] = location
            # ロケーションのタイプに合わせてタイルを設定
            if location.loc_type == 'town':
                self.set_tile(location.row, location.col, TILE_TOWN)
            elif location.loc_type == 'dungeon':
                self.set_tile(location.row, location.col, TILE_DUNGEON)
            elif location.loc_type == 'castle':
                self.set_tile(location.row, location.col, TILE_CASTLE)
            return True
        return False
    
    def get_location(self, row: int, col: int) -> Optional[Location]:
        """特定の位置のロケーションを取得
        
        Args:
            row: 行位置
            col: 列位置
            
        Returns:
            ロケーションオブジェクト（ない場合はNone）
        """
        key = (row, col)
        return self.locations.get(key)
    
    def display(self, player_pos: Tuple[int, int] = None):
        """マップを表示
        
        Args:
            player_pos: プレイヤーの位置（行, 列）
        """
        plt.figure(figsize=(10, 8))
        
        # マップをカラーコードとして表示
        cmap = plt.matplotlib.colors.ListedColormap(
            [TILE_COLORS[i] for i in range(max(TILE_COLORS.keys()) + 1)]
        )
        plt.imshow(self.tiles, cmap=cmap, interpolation='nearest')
        
        # 座標を表示
        plt.grid(True, color='black', alpha=0.2)
        plt.xticks(np.arange(0, self.width, 1))
        plt.yticks(np.arange(0, self.height, 1))
        
        # ロケーションに印をつける
        for (row, col), location in self.locations.items():
            plt.text(col, row, 'L', color='white', fontsize=12, ha='center', va='center')
        
        # プレイヤーの位置を表示
        if player_pos:
            player_row, player_col = player_pos
            plt.plot(player_col, player_row, 'ro', markersize=10)  # 赤い円でプレイヤーを表示
        
        plt.title(f"マップ: {self.name}")
        plt.xlabel("列 (X)")
        plt.ylabel("行 (Y)")
        plt.tight_layout()
        plt.show()
    
    def save_to_file(self, filepath: str) -> bool:
        """マップデータをJSONファイルに保存
        
        Args:
            filepath: 保存先のファイルパス
            
        Returns:
            保存に成功したかどうか
        """
        try:
            data = {
                'name': self.name,
                'width': self.width,
                'height': self.height,
                'tiles': self.tiles.tolist(),
                'locations': {
                    f"{row},{col}": loc.to_dict() 
                    for (row, col), loc in self.locations.items()
                }
            }
            
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
            return True
        except Exception as e:
            print(f"マップの保存中にエラーが発生しました: {e}")
            return False
    
    @classmethod
    def load_from_file(cls, filepath: str) -> Optional['GameMap']:
        """JSONファイルからマップデータを読み込み
        
        Args:
            filepath: 読み込むファイルパス
            
        Returns:
            GameMapオブジェクト、読み込みに失敗した場合はNone
        """
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
            game_map = cls(data['name'], data['width'], data['height'])
            game_map.tiles = np.array(data['tiles'])
            
            # ロケーション情報の復元
            for key, loc_data in data['locations'].items():
                row, col = map(int, key.split(','))
                location = Location(
                    name=loc_data['name'],
                    description=loc_data.get('description', ''),
                    loc_type=loc_data['type'],
                    row=row,
                    col=col,
                    events=loc_data.get('events', [])
                )
                location.visited = loc_data.get('visited', False)
                game_map.locations[(row, col)] = location
                
            return game_map
        except Exception as e:
            print(f"マップの読み込み中にエラーが発生しました: {e}")
            return None

# プレイヤーのマップ上の状態を管理するクラス
class MapPlayer:
    """マップ上でのプレイヤーの状態を管理するクラス"""
    
    def __init__(self, character: PlayerCharacter, row: int, col: int):
        """マッププレイヤーの初期化
        
        Args:
            character: プレイヤーキャラクター
            row: 初期位置の行
            col: 初期位置の列
        """
        self.character = character
        self.row = row
        self.col = col
        self.movement_points = 100  # 移動ポイント（1日の行動力）
        self.steps = 0  # 歩数カウント
        
    def move(self, direction: int, game_map: GameMap) -> Tuple[bool, Optional[Location], bool]:
        """指定方向に移動
        
        Args:
            direction: 移動方向
            game_map: 現在のマップ
            
        Returns:
            (移動成功したか, 到着したロケーション, エンカウントが発生したか)
        """
        if direction not in DIRECTION_CHANGES:
            return False, None, False
        
        # 移動先の座標を計算
        row_change, col_change = DIRECTION_CHANGES[direction]
        new_row = self.row + row_change
        new_col = self.col + col_change
        
        # 移動先が通行可能か確認
        if not game_map.is_walkable(new_row, new_col):
            return False, None, False
        
        # 移動コストを計算
        tile_type = game_map.get_tile(new_row, new_col)
        move_cost = TILE_MOVEMENT_COST[tile_type]
        
        # 移動ポイントが足りるか確認
        if self.movement_points < move_cost:
            return False, None, False
        
        # 移動を実行
        self.row = new_row
        self.col = new_col
        self.movement_points -= move_cost
        self.steps += 1
        
        # 到着したロケーションを取得
        location = game_map.get_location(self.row, self.col)
        
        # エンカウント判定
        encounter_rate = TILE_ENCOUNTER_RATE[tile_type]
        encounter = random.random() < encounter_rate
        
        return True, location, encounter
    
    def rest(self):
        """休息して移動ポイントを回復"""
        self.movement_points = 100  # 1日の移動ポイントをリセット
        # キャラクターのHP/MPも回復する
        self.character.current_hp = self.character.max_hp
        self.character.current_mp = self.character.max_mp

def create_sample_map() -> GameMap:
    """サンプルマップを作成
    
    Returns:
        作成したGameMapオブジェクト
    """
    # 10x7のマップを作成（元は20x15）
    game_map = GameMap("はじまりの大地", 10, 7)
    
    # 基本的な地形を設定
    for row in range(game_map.height):
        for col in range(game_map.width):
            # デフォルトは平地
            game_map.set_tile(row, col, TILE_EMPTY)
            
    # 森を配置（数を調整）
    for _ in range(10):  # 元は30
        row = random.randint(0, game_map.height - 1)
        col = random.randint(0, game_map.width - 1)
        game_map.set_tile(row, col, TILE_FOREST)
    
    # 山を配置（数を調整）
    for _ in range(5):  # 元は15
        row = random.randint(0, game_map.height - 1)
        col = random.randint(0, game_map.width - 1)
        game_map.set_tile(row, col, TILE_MOUNTAIN)
    
    # 水を配置（川を作る）- 位置調整
    river_start_col = random.randint(3, 6)  # 元は5, 15
    for row in range(game_map.height):
        river_width = random.randint(1, 1)  # 元は1, 2（幅を小さく）
        for i in range(river_width):
            game_map.set_tile(row, river_start_col + i, TILE_WATER)
        river_start_col += random.randint(-1, 1)
        river_start_col = max(1, min(game_map.width - 2, river_start_col))
    
    # 草原を配置（数を調整）
    for _ in range(15):  # 元は40
        row = random.randint(0, game_map.height - 1)
        col = random.randint(0, game_map.width - 1)
        if game_map.get_tile(row, col) == TILE_EMPTY:
            game_map.set_tile(row, col, TILE_GRASS)
    
    # 道を配置（東西に伸びる道）
    road_row = random.randint(3, game_map.height - 2)
    for col in range(game_map.width):
        if game_map.get_tile(road_row, col) != TILE_WATER:
            game_map.set_tile(road_row, col, TILE_ROAD)
    
    # 南北に伸びる道
    road_col = random.randint(2, game_map.width - 2)
    for row in range(game_map.height):
        if game_map.get_tile(row, road_col) != TILE_WATER:
            game_map.set_tile(row, road_col, TILE_ROAD)
    
    # ロケーションを追加（位置調整）
    # 町
    town = Location(
        name="始まりの村",
        description="この地方で最も古い村。多くの冒険者がここから旅立つ。",
        loc_type="town",
        row=road_row,
        col=1,  # 元は3
        events=[
            {
                "type": "shop",
                "name": "村の雑貨屋",
                "description": "日用品や冒険の道具を売っている"
            },
            {
                "type": "inn",
                "name": "村の宿屋",
                "description": "安価で泊まれる宿。体力が全回復する"
            }
        ]
    )
    game_map.add_location(town)
    
    # 別の町
    town2 = Location(
        name="川向こうの町",
        description="川の東側にある小さな町。釣り師が多い。",
        loc_type="town",
        row=road_row,
        col=8,  # 元は16
        events=[
            {
                "type": "shop",
                "name": "釣り具屋",
                "description": "釣り道具を専門に扱う店"
            }
        ]
    )
    game_map.add_location(town2)
    
    # ダンジョン
    dungeon = Location(
        name="古い洞窟",
        description="村の北にある不気味な洞窟。モンスターが出ると言われている。",
        loc_type="dungeon",
        row=1,  # 元は2
        col=road_col,
        events=[
            {
                "type": "battle",
                "name": "洞窟の奥",
                "description": "洞窟の奥でスライムの群れに遭遇した！"
            }
        ]
    )
    game_map.add_location(dungeon)
    
    # 城
    castle = Location(
        name="王城",
        description="この地方を治める王の城。立派な建物が建っている。",
        loc_type="castle",
        row=5,  # 元は12
        col=road_col,
        events=[
            {
                "type": "quest",
                "name": "王様の依頼",
                "description": "王様から魔物退治の依頼を受けた"
            }
        ]
    )
    game_map.add_location(castle)
    
    return game_map
# 画像ディレクトリの設定
import os
from typing import Any
image_dir = os.path.join('images', 'tiles')

# 画像がなければディレクトリを作成
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
    print(f"'{image_dir}'ディレクトリを作成しました。")

# タイル画像の設定（後で実際の画像ファイルに置き換えることができます）
TILE_IMAGES = {
    TILE_EMPTY: os.path.join(image_dir, 'empty.png'),
    TILE_WALL: os.path.join(image_dir, 'wall.png'),
    TILE_GRASS: os.path.join(image_dir, 'grass.png'),
    TILE_FOREST: os.path.join(image_dir, 'forest.png'),
    TILE_WATER: os.path.join(image_dir, 'water.png'),
    TILE_MOUNTAIN: os.path.join(image_dir, 'mountain.png'),
    TILE_ROAD: os.path.join(image_dir, 'road.png'),
    TILE_TOWN: os.path.join(image_dir, 'town.png'),
    TILE_DUNGEON: os.path.join(image_dir, 'dungeon.png'),
    TILE_CASTLE: os.path.join(image_dir, 'castle.png')
}

def generate_tile_images():
    """タイル画像が存在しない場合、シンプルな画像を生成する"""
    import matplotlib.pyplot as plt
    import numpy as np
    
    # 各タイルタイプの色でシンプルな画像を生成
    for tile_type, img_path in TILE_IMAGES.items():
        if not os.path.exists(img_path):
            # タイルサイズ設定
            size = 64
            
            # 画像の作成
            img = np.ones((size, size, 3))
            color = TILE_COLORS[tile_type]
            
            # 16進カラーコードをRGB値に変換
            r = int(color[1:3], 16) / 255.0
            g = int(color[3:5], 16) / 255.0
            b = int(color[5:7], 16) / 255.0
            
            img[:, :, 0] = r
            img[:, :, 1] = g
            img[:, :, 2] = b
            
            # 特殊タイル用のシンボルを追加
            if tile_type == TILE_TOWN:
                # 町用のシンプルな家のシンボル
                img[20:40, 20:40, :] = 0.7
                img[15:25, 25:35, :] = 0.5
            elif tile_type == TILE_CASTLE:
                # 城用のシンプルな城のシンボル
                img[10:50, 20:44, :] = 0.7  # 本体
                img[5:15, 15:49, :] = 0.5   # 屋根
                img[15:50, 25:30, :] = 0.3  # ドア
                img[5:15, 20:25, :] = 0.3   # 左の塔
                img[5:15, 39:44, :] = 0.3   # 右の塔
            elif tile_type == TILE_DUNGEON:
                # ダンジョン用の洞窟シンボル
                img[20:45, 15:50, :] = 0.2
            elif tile_type == TILE_FOREST:
                # 森用の木のシンボル
                img[15:45, 25:40, :] = 0.3  # 幹
                img[5:25, 15:50, :] = 0.1   # 葉
            elif tile_type == TILE_MOUNTAIN:
                # 山用の三角形
                for i in range(size):
                    for j in range(size):
                        if i > size/2 - abs(j - size/2)/2:
                            img[i, j, :] = [0.4, 0.3, 0.2]
            
            # 画像の保存
            plt.imsave(img_path, img)
            print(f"タイル画像を生成しました: {img_path}")

# タイル画像の生成（必要な場合）
generate_tile_images()

# GameMapクラスのdisplayメソッドを改良
def display_map_with_images(self, player_pos=None):
    """マップを画像タイルで表示
    
    Args:
        player_pos: プレイヤーの位置（行, 列）
    """
    import matplotlib.pyplot as plt
    from matplotlib.offsetbox import OffsetImage, AnnotationBbox
    import matplotlib.image as mpimg
    
    # 日本語フォントの設定
    try:
        # フォント設定を試みる
        plt.rcParams['font.family'] = 'sans-serif'
        # 英語のフォントでタイトルを表示
        map_title = "Map: " + self.name
    except:
        # 英語のフォールバックタイトル
        map_title = "Map"
    
    # figureサイズを小さくする
    plt.figure(figsize=(7, 5))
    ax = plt.gca()
    
    # 背景色を設定
    ax.set_facecolor('#f0f0f0')
    
    # マップの範囲を設定
    ax.set_xlim(-0.5, self.width - 0.5)
    ax.set_ylim(self.height - 0.5, -0.5)  # Y軸は反転
    
    # グリッド線を表示
    ax.grid(True, color='gray', alpha=0.3, linestyle='-')
    
    # タイルの表示
    for row in range(self.height):
        for col in range(self.width):
            tile_type = self.get_tile(row, col)
            
            # タイル画像のパス
            img_path = TILE_IMAGES[tile_type]
            
            try:
                # 画像の読み込み
                if os.path.exists(img_path):
                    img = mpimg.imread(img_path)
                    # ズーム値を小さくする
                    imagebox = OffsetImage(img, zoom=0.18)
                    ab = AnnotationBbox(imagebox, (col, row), frameon=False)
                    ax.add_artist(ab)
                else:
                    # 画像がない場合は色で表示
                    rect = plt.Rectangle((col - 0.5, row - 0.5), 1, 1, 
                                        facecolor=TILE_COLORS[tile_type])
                    ax.add_patch(rect)
            except Exception as e:
                print(f"タイル画像の読み込みエラー: {e}")
                # エラーが出たら色で表示
                rect = plt.Rectangle((col - 0.5, row - 0.5), 1, 1, 
                                    facecolor=TILE_COLORS[tile_type])
                ax.add_patch(rect)
    
    # ロケーションの表示（城、町、ダンジョンなど）
    for (row, col), location in self.locations.items():
        if location.loc_type == 'town':
            marker = 'o'
            color = 'blue'
            label = 'T'
        elif location.loc_type == 'dungeon':
            marker = 's'
            color = 'purple'
            label = 'D'
        elif location.loc_type == 'castle':
            marker = '^'
            color = 'gold'
            label = 'C'
        else:
            marker = 'x'
            color = 'black'
            label = 'L'
        
        # マーカーの描画
        ax.plot(col, row, marker=marker, markersize=8, 
               markerfacecolor=color, markeredgecolor='black')
        
        # ラベルの描画
        ax.text(col, row, label, color='white', ha='center', va='center',
               fontweight='bold', fontsize=6)
    
    # プレイヤーの位置を表示
    if player_pos:
        player_row, player_col = player_pos
        # プレイヤーのシンボル
        ax.plot(player_col, player_row, 'o', markersize=10, 
                markerfacecolor='red', markeredgecolor='black', zorder=10)
        ax.text(player_col, player_row, 'P', color='white', ha='center', va='center',
                fontweight='bold', fontsize=6, zorder=11)
    
    # 座標ラベルを設定
    ax.set_xticks(range(self.width))
    ax.set_yticks(range(self.height))
    
    # 英語タイトルに変更
    plt.title(map_title, fontsize=12)
    plt.tight_layout(pad=0.5)
    plt.show()

def display_map_with_images(self, player_pos=None):
    """マップを画像タイルで表示
    
    Args:
        player_pos: プレイヤーの位置（行, 列）
    """
    import matplotlib.pyplot as plt
    from matplotlib.offsetbox import OffsetImage, AnnotationBbox
    import matplotlib.image as mpimg
    
    # デフォルトのタイトルを設定
    map_title = "Map: World Map"
    
    # 日本語フォント設定を試みる
    try:
        # 一般的な日本語フォントを試す
        available_fonts = ['Noto Sans CJK JP', 'Hiragino Sans', 'Yu Gothic', 'MS Gothic', 'Meiryo']
        for font in available_fonts:
            try:
                plt.rcParams['font.family'] = font
                # 日本語タイトルを設定
                map_title = f"マップ: {self.name}"
                break
            except:
                continue
    except:
        # エラーが発生した場合は英語タイトルを使用
        pass
    
    # figureサイズを調整
    plt.figure(figsize=(5, 3))
    ax = plt.gca()
    
    # 背景色を設定
    ax.set_facecolor('#f0f0f0')
    
    # マップの範囲を設定
    ax.set_xlim(-0.5, self.width - 0.5)
    ax.set_ylim(self.height - 0.5, -0.5)  # Y軸は反転
    
    # グリッド線を表示
    ax.grid(True, color='gray', alpha=0.3, linestyle='-')
    
    # タイルの表示
    for row in range(self.height):
        for col in range(self.width):
            tile_type = self.get_tile(row, col)
            
            # タイル画像のパス
            img_path = TILE_IMAGES[tile_type]
            
            try:
                # 画像の読み込み
                if os.path.exists(img_path):
                    img = mpimg.imread(img_path)
                    # ズーム値を小さくする
                    imagebox = OffsetImage(img, zoom=0.15)
                    ab = AnnotationBbox(imagebox, (col, row), frameon=False)
                    ax.add_artist(ab)
                else:
                    # 画像がない場合は色で表示
                    rect = plt.Rectangle((col - 0.5, row - 0.5), 1, 1, 
                                        facecolor=TILE_COLORS[tile_type])
                    ax.add_patch(rect)
            except Exception as e:
                # エラーが出たら色で表示
                rect = plt.Rectangle((col - 0.5, row - 0.5), 1, 1, 
                                    facecolor=TILE_COLORS[tile_type])
                ax.add_patch(rect)
    
    # ロケーションの表示（城、町、ダンジョンなど）
    for (row, col), location in self.locations.items():
        if location.loc_type == 'town':
            marker = 'o'
            color = 'blue'
            label = 'T'
        elif location.loc_type == 'dungeon':
            marker = 's'
            color = 'purple'
            label = 'D'
        elif location.loc_type == 'castle':
            marker = '^'
            color = 'gold'
            label = 'C'
        else:
            marker = 'x'
            color = 'black'
            label = 'L'
        
        # マーカーの描画
        ax.plot(col, row, marker=marker, markersize=8, 
               markerfacecolor=color, markeredgecolor='black')
        
        # ラベルの描画
        ax.text(col, row, label, color='white', ha='center', va='center',
               fontweight='bold', fontsize=6)
    
    # プレイヤーの位置を表示
    if player_pos:
        player_row, player_col = player_pos
        # プレイヤーのシンボル
        ax.plot(player_col, player_row, 'o', markersize=10, 
                markerfacecolor='red', markeredgecolor='black', zorder=10)
        ax.text(player_col, player_row, 'P', color='white', ha='center', va='center',
                fontweight='bold', fontsize=6, zorder=11)
    
    # 座標ラベルを設定
    ax.set_xticks(range(self.width))
    ax.set_yticks(range(self.height))
    
    # タイトル
    plt.title(map_title, fontsize=12)
    plt.tight_layout(pad=0.5)
    plt.show()

def display_minimap(self, player_pos, view_range=2):  # 元は3
    """プレイヤー周辺のミニマップを表示
    
    Args:
        player_pos: プレイヤーの位置（行, 列）
        view_range: 表示範囲（プレイヤーの位置から何マス見えるか）
    """
    import matplotlib.pyplot as plt
    
    if not player_pos:
        print("プレイヤーの位置が指定されていません。")
        return
    
    player_row, player_col = player_pos
    
    # 表示範囲の計算
    min_row = max(0, player_row - view_range)
    max_row = min(self.height - 1, player_row + view_range)
    min_col = max(0, player_col - view_range)
    max_col = min(self.width - 1, player_col + view_range)
    
    # 表示幅と高さ
    display_height = max_row - min_row + 1
    display_width = max_col - min_col + 1
    
    plt.figure(figsize=(3, 3))  # 元は(5, 5)
    ax = plt.gca()
    
    # 領域の設定
    ax.set_xlim(min_col - 0.5, max_col + 0.5)
    ax.set_ylim(max_row + 0.5, min_row - 0.5)  # Y軸は反転
    
    # タイルの表示
    for row in range(min_row, max_row + 1):
        for col in range(min_col, max_col + 1):
            tile_type = self.get_tile(row, col)
            rect = plt.Rectangle((col - 0.5, row - 0.5), 1, 1, 
                                 facecolor=TILE_COLORS[tile_type])
            ax.add_patch(rect)
            
            # タイルの種類を文字で表示（オプション）
            if tile_type == TILE_TOWN:
                ax.text(col, row, 'T', ha='center', va='center', fontsize=8)
            elif tile_type == TILE_CASTLE:
                ax.text(col, row, 'C', ha='center', va='center', fontsize=8)
            elif tile_type == TILE_DUNGEON:
                ax.text(col, row, 'D', ha='center', va='center', fontsize=8)
    
    # プレイヤーの位置
    ax.plot(player_col, player_row, 'o', markersize=10, 
            markerfacecolor='red', markeredgecolor='black')
    
    # グリッド線
    ax.grid(True, color='black', alpha=0.2)
    
    plt.title("Mini Map", fontsize=12)  # 英語タイトルに変更
    plt.tight_layout()
    plt.show()
    
# GameMapクラスにメソッドを追加
GameMap.display_with_images = display_map_with_images
GameMap.display_minimap = display_minimap

# マップシステムの基本機能のみを提供
def initialize_map_system():
    """マップシステムの初期化"""
    # サンプルマップを作成
    game_map = create_sample_map()
    print(f"マップシステムを初期化: '{game_map.name}' ({game_map.width}x{game_map.height})")
    return game_map

# 最後に追加
if __name__ == "__main__":
    print("マップシステムを読み込みました。main.ipynbから使用できます。")

Exception: File `'notebooks/characters.ipynb'` not found.